### Задание 7

In [161]:
import pandas as pd


dfe = pd.read_excel('../data/документация/КТ_Кировский.xlsx')
dfc = pd.read_csv('../data/документация/sp.csv')

dfec = dfe.copy(deep=True)
dfcc = dfc.copy(deep=True)

dfcc['Сумма'] = dfcc['Сумма'].replace(',', '.', regex=True).astype(float)
dfcc['Пени'] = dfcc['Пени'].replace(',', '.', regex=True).astype(float)
dfcc['ГосПошлина'] = dfcc['ГосПошлина'].replace(',', '.', regex=True).astype(float)

#dfcc['Общий долг'] = dfcc[['Сумма', 'Пени', 'ГосПошлина']].sum(axis=1)

In [178]:
df = dfcc.copy(deep=True)
df['Номер ИД'] = df['Номер ИД'].str.split(r',\s*')
df = df.explode('Номер ИД')

df[df['Номер ИД'] == "2-88-1941/2022"], df[df['Номер ИД'] == "2-88-1942/2022"]


(              Номер ИД       Номер       Счет      Сумма     Пени  ГосПошлина
 377816  2-88-1941/2022  2148135485  3278559.0  128377.34  2755.62         0.0
 377817  2-88-1941/2022  2148135485  3278559.0  128377.34  9439.56      1023.2
 377818  2-88-1941/2022  2148135485  3278559.0  128377.34  9439.56      1023.2,
               Номер ИД       Номер       Счет      Сумма     Пени  ГосПошлина
 377818  2-88-1942/2022  2148135485  3278559.0  128377.34  9439.56      1023.2)

In [107]:
merged_df = pd.merge(dfec, dfcc[['Номер ИД', 'Сумма', 'Пени', 'ГосПошлина']], how='left').drop_duplicates()
merged_df['Общий долг'] = merged_df[['Сумма', 'Пени', 'ГосПошлина']].sum(axis=1)

print(f"Null - {merged_df[merged_df['Общий долг'].isna()].shape}, All - {merged_df.shape}")


Null - (0, 35), All - (2520, 35)


In [ ]:
pd.set_option('display.max_columns', None)
merged_df[merged_df['Общий долг'] == 0]

In [108]:
merged_df.to_excel('../data/документация/КТ_Кировский_ОбщийДолг.xlsx')

In [135]:
dfcc.dtypes

Номер ИД       object
Номер           int64
Счет          float64
Сумма         float64
Пени          float64
ГосПошлина    float64
dtype: object